In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans
import warnings
import time
from tqdm import tqdm
import pylab
from sklearn.decomposition import PCA
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import warnings
warnings.filterwarnings('ignore')

In [2]:
# df = pd.read_csv('Reviews.csv') # only do once

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
path = '/content/drive/MyDrive/Colab Notebooks/ITC/Final Project/data.npy' # folder path
# np.save(path, df) # saves df to this path so only do once

data = np.load(path, allow_pickle = True) # loads in an array

In [5]:
df = pd.DataFrame(data, columns = ['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'])

##### *Train-Test Split*

In [6]:
from sklearn.model_selection import GroupShuffleSplit
splitter_temp = GroupShuffleSplit(test_size=.40, n_splits=1, random_state = 42)
split_temp = splitter_temp.split(df, groups=df['ProductId'])
train_inds, temp_inds = next(split_temp)

train = df.iloc[train_inds]
temp = df.iloc[temp_inds]

In [7]:
splitter_val = GroupShuffleSplit(test_size=.50, n_splits=1, random_state = 42)
split_val = splitter_val.split(temp, groups=temp['ProductId'])
val_inds, test_inds = next(split_val)

val = temp.iloc[val_inds]
test = temp.iloc[test_inds]

In [8]:
num_samples = 50000

# Split train and test sets
X_train = train[:num_samples].drop(columns = 'Score')
y_train = train[:num_samples].Score

X_test = test[:num_samples].drop(columns = 'Score')
y_test = test[:num_samples].Score

In [9]:
y_train_df = pd.DataFrame(y_train)
full_train = pd.concat([X_train, y_train_df], axis=1)

train_negative = full_train[full_train['Score'].isin([1, 2])]
train_neutral = full_train[full_train['Score'] == 3]
train_positive = full_train[full_train['Score'].isin([4, 5])]

corpus_full = full_train['Text'].tolist()
corpus_negative = train_negative['Text'].tolist()
corpus_neutral = train_neutral['Text'].tolist()
corpus_positive = train_positive['Text'].tolist()

In [10]:
from collections import Counter, defaultdict
import spacy
nlp = spacy.load("en_core_web_sm")

## BERTopic

In [ ]:
!pip install bertopic

In [12]:
from bertopic import BERTopic

*Negative Reviews*

In [13]:
# Negative reviews
docs = corpus_negative
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(docs)

# Get detailed information about each topic found by the BERTopic model
topic_info = topic_model.get_topic_info()

# Display the first 7 rows of the topic information, focusing on specific columns
filtered_topic_info = topic_info.head(7).set_index('Topic')[['Count', 'Name', 'Representation']]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
# Visualize the top 16 topics as a bar chart, showing the 10 most relevant words for each topic
topic_model.visualize_barchart(top_n_topics=16, n_words=10)

*Positive Reviews*

In [21]:
# Positive reviews
docs = corpus_positive
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(docs)

# Get detailed information about each topic found by the BERTopic model
topic_info = topic_model.get_topic_info()

# Display the first 7 rows of the topic information, focusing on specific columns
filtered_topic_info = topic_info.head(7).set_index('Topic')[['Count', 'Name', 'Representation']]

# Get detailed information about each topic found by the BERTopic model
topic_info = topic_model.get_topic_info()

# Display the first 7 rows of the topic information, focusing on specific columns
filtered_topic_info = topic_info.head(7).set_index('Topic')[['Count', 'Name', 'Representation']]

In [22]:
# Visualize the top 16 topics as a bar chart, showing the 10 most relevant words for each topic
topic_model.visualize_barchart(top_n_topics=16, n_words=10)

*All Reviews*

In [19]:
# All reviews
docs = corpus_full
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(docs)

# Get detailed information about each topic found by the BERTopic model
topic_info = topic_model.get_topic_info()

# Display the first 7 rows of the topic information, focusing on specific columns
filtered_topic_info = topic_info.head(7).set_index('Topic')[['Count', 'Name', 'Representation']]

# Get detailed information about each topic found by the BERTopic model
topic_info = topic_model.get_topic_info()

# Display the first 7 rows of the topic information, focusing on specific columns
filtered_topic_info = topic_info.head(7).set_index('Topic')[['Count', 'Name', 'Representation']]

In [20]:
# Visualize the top 16 topics as a bar chart, showing the 10 most relevant words for each topic
topic_model.visualize_barchart(top_n_topics=16, n_words=10)